In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
%reload_ext autoreload
%autoreload 2
%matplotlib inline

Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import pandas as pd
import seaborn as sns

from postprocessor.core.processes.fft import fft, fftParameters

# Load data

In [ ]:
data_dir = "../data/raw/"
group_name = "st01649_tsa1tsa2morgan"

In [ ]:
filepath = data_dir + group_name
timeseries_filepath = filepath + "_flavin_timeseries.csv"
labels_filepath = filepath + "_labels.csv"

timeseries_df = pd.read_csv(timeseries_filepath, index_col=[0,1,2])
labels_df = pd.read_csv(labels_filepath, index_col=[0,1,2])

# Select data

Drop NaNs

In [ ]:
timeseries_dropna = timeseries_df.dropna()

In [ ]:
labels_dropna = labels_df.loc[timeseries_dropna.index]

Select oscillatory time series

In [ ]:
timeseries_osc = timeseries_dropna.loc[labels_dropna[labels_dropna.score == 1].index]

In [ ]:
timeseries_osc

# Signal-to-noise ratio

In [ ]:
fft_freqs_df, fft_power_df = fft.as_function(timeseries_osc)

In [ ]:
def find_nearest(array, value):
    """find index of nearest value in numpy array"""
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


def get_snr(fft_freqs_df, fft_power_df, cutoff_freq):
    """Get signal-to-noise ratio from a Fourier spectrum

    Get signal-to-noise ratio from a Fourier spectrum. Defines a cut-off
    frequency; frequencies lower than this is considered signal, while
    frequencies higher than this is considered noise. The signal-to-noise
    ratio is defined as the area under the Fourier spectrum to the left of
    the cut-off divided by the area under the Fourier spectrum to the right
    of the cut-off. Follows:

    Parameters
    ----------
    fft_freqs_df : pandas.DataFrame
        DataFrame showing in each row the frequency dimension of each
        Fourier spectrum
    fft_power_df : pandas.DataFrame
        DataFrame showing in each row the periodogram (Fourier spectrum)
    cutoff_freq : float
        cut-off frequency to divide signal and noise
    """
    fft_freqs_array = fft_freqs_df.to_numpy()
    fft_power_array = fft_power_df.to_numpy()
    snr = []
    for rowindex, _ in enumerate(fft_power_array):
        cutoff_freq_nearest = find_nearest(
            fft_freqs_array[rowindex, :], cutoff_freq
        )
        # nans can occur if the origin time series has nans -- skip over these
        if np.isnan(cutoff_freq_nearest):
            snr.append(np.nan)
        else:
            cutoff_colindex = np.where(
                fft_freqs_array[rowindex, :] == cutoff_freq_nearest
            )[0].item()
            area_all = np.trapz(
                y=fft_power_array[rowindex, :], x=fft_freqs_array[rowindex, :]
            )
            area_signal = np.trapz(
                y=fft_power_array[rowindex, 0:cutoff_colindex],
                x=fft_freqs_array[rowindex, 0:cutoff_colindex],
            )
            area_noise = area_all - area_signal
            snr.append(area_signal / area_noise)
    return np.array(snr)

In [ ]:
snr_array = get_snr(fft_freqs_df, fft_power_df, cutoff_freq=0.01766784)

In [ ]:
from postprocessor.routines.histogram import histogram

In [ ]:
histogram(
    snr_array,
    label='snr',
    binsize=0.5,
)

In [ ]:
np.median(snr_array)